<div class="frontmatter text-center">
<h1> Introduction to Data Science and Programming</h1>
<h2>Exercise 13: Normal distribution</h2>
<h3>IT University of Copenhagen, Fall 2023</h3>
</div>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import re
import scipy.stats as stats

# A Data Riddle

The Riddler is a weekly column at [fivethirtyeight](https://fivethirtyeight.com/), where they offer puzzles on maths, logic and probability. These puzzles are often solved using data analysis approaches. Here we focus on the normal distribution.

The following puzzle could originally be found [here](https://fivethirtyeight.com/features/are-you-the-best-warlord/). The puzzle received submissions from around 1500 users, and their submissions are available [here](https://github.com/fivethirtyeight/data/blob/master/riddler-castles/castle-solutions-3.csv).

___
#### Are you The Best Warlord?

In a distant, war-torn land, there are two warlords: you and your archenemy.

In this land, there are ten castles. Each castle has its own strategic value for a would-be conqueror. Specifically, the castles are worth 1, 2, 3, …, 9, and 10 victory points (corresponding to the castle number). This means there are 55 victory points in total up for grabs points in the war.

You and your enemy each have 100 soldiers to distribute, any way you like, to fight at any of the ten castles. Whoever sends more soldiers to a given castle conquers that castle and wins its victory points. If you each send the same number of troops, you split the points. 

You don’t know what distribution of forces your enemy has chosen until the battles begin. Whoever wins the most points wins the war.
___

In the exercises for today you will be working with these submitted solutions to the puzzle, which can be found in `submissions.txt`.

The file has the following variables:

| Column index | Variable | Definition |
| -:- | :- | :- | 
| 0 | Castle 1 | The number of troops deployed to the castle |
| 1 | Castle 2 | The number of troops deployed to the castle |
| 2 | Castle 3 | The number of troops deployed to the castle |
| 3 | Castle 4 | The number of troops deployed to the castle |
| 4 | Castle 5 | The number of troops deployed to the castle |
| 5 | Castle 6 | The number of troops deployed to the castle |
| 6 | Castle 7 | The number of troops deployed to the castle |
| 7 | Castle 8 | The number of troops deployed to the castle |
| 8 | Castle 9 | The number of troops deployed to the castle |
| 9 | Castle 10 | The number of troops deployed to the castle |
| 10 | Why did you choose your troop deployment? | String containing the reasoning behind the submission |


### 13.0. Load the data and convert it to a numpy array
Inspect the `submissions.txt` file and read through the provided function below to understand what each line does

Once you've done that, apply the function to the `submissions.txt` file and convert the resulting data structure into a numpy array which you will use for the rest of the exercises.

In [ ]:
def read_csv(filepath):
    output = []
    regex = "^\d+,\d+,\d+,\d+,\d+,\d+,\d+,\d+,\d+,\d+"
    with open(filepath, "r", errors="replace") as file:
        lines = file.readlines()
    for line in lines:
        regex_find = re.search(regex,line)
        if regex_find:
            line = regex_find.group() 
            line = line.split(",") 
            line = [int(value) for value in line]
            output.append(line)
    return output

In [ ]:
data = np.array(read_csv('files/submissions.txt'))
data.shape

### 13.1. Clean the data

We know that there are supposed to be 100 deployed troops in total per submission. This is not the case for all of the submissions, however, so we need to clean the data.

Extract the rows where the sum over all the castles of the row is 100 and store your cleaned data in a new numpy array. 
___

Clean data array shape: `(1321,10)`

In [ ]:
data = data[np.sum(data, axis=1)==100, :]
data.shape

### 13.2. Find the mean and standard deviation of each castle's number of deployed troops
Hint: Check the jupyter notebook from today's lecture.

For each castle, fit the distribution as a normal distribution (using `stats.norm.fit`) and obtain its mean, `mu`, and standard deviation, `sigma`. 
___

For `Castle 1`, at column index 0, you should get mu `2.025` and sigma `3.875`.

In [ ]:
# fits = [stats.norm.fit(data[:, i]) for i in range(data.shape[1])]
fits = np.array(list((zip(np.mean(data, axis=0), np.std(data, axis=0))))) # double the speed
fits

### 13.3. Plot histogram and corresponding normal distribution
Hint: Check the lecture jupyter notebook

For the castle (column) with the **highest mean value**, plot the histogram and the corresponding normal distribution. This normal distribution can be obtained using `stats.norm.fit()`.

In [ ]:
max_mean_castle = np.argmax(fits[:, 0])

def plot_dist(castle_id):
    fig, ax = plt.subplots()
    ax.hist(data[:, castle_id], bins=25, density=True);
    xs = np.linspace(min(data[:, castle_id]), max(data[:, castle_id]), 100)
    ax.plot(xs, stats.norm.pdf(xs, fits[castle_id, 0], fits[castle_id, 1]), c='red')
    ax.set_title('Troop Deployment Distribution');
    ax.set_xlabel('Number of Troops deployed');
    ax.set_ylabel('Frequency');

plot_dist(max_mean_castle)

### 13.4. Plot histogram and corresponding normal distribution
Hint: Check the lecture jupyter notebook

For the castle (column) with the **lowest standard deviation value**, plot the histogram and the corresponding normal distribution. This normal distribution can be obtained using `stats.norm.fit()`.

In [ ]:
min_std_castle = np.argmin(fits[:, 1])

plot_dist(min_std_castle)

### 13.5. Plot histogram and corresponding normal distribution
Hint: Check the lecture jupyter notebook

For the castle (column) with the **highest standard deviation value**, plot the histogram and the corresponding normal distribution. This normal distribution can be obtained using `stats.norm.fit()`.

In [ ]:
max_std_castle = np.argmax(fits[:, 1])

plot_dist(max_std_castle)

### 13.6. Check if the above data distributions are good fits as normal distribution 
Hint: Check the lecture jupyter notebook

Plot the Q-Q plots for the three castles from the prior exercises and verify, and if the above data distributions are good fits as normal distribution.

In [ ]:
import statsmodels.api as sm

In [ ]:
fig, ax = plt.subplots(ncols=3, figsize=(6*3, 4))
for i, x in enumerate([max_mean_castle, min_std_castle, max_std_castle]):
    sm.qqplot(data[:, x], dist="norm", ax=ax[i], line='45')
    ax[i].set_title(f'QQ-Plot for Normal fit on Castle {x+1}')

### 13.7. Find the mode(s) and median for each of the 10 castles

Hint: 
- https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.mode.html
- https://numpy.org/doc/stable/reference/generated/numpy.median.html

___
Mode: `[[ 1,  1,  1,  1,  1,  0,  0, 20,  0,  0]]`
Median: `[ 1.  2.  3.  6.  9. 12. 14. 18. 18. 14.]`

In [ ]:
print(list(stats.mode(data, axis=0))[0][0])
print(np.median(data, axis=0).astype(int))

### 13.8. Calculate the scores for all of the potential 'battles' of each player
For each submission (row), compare it with every other row and calculate the score for each player in each of the simulated 'battles'.

The result will be 1321x1321 matrix. Each value at position (i,j) in the matrix will represent the score of player i in the battle against player j. So the first value in the first row is the result of submission 0 against itself, the second value in the first row is the result of submission 0 against submission 1, etc.

Recall the rules:

- The castles are worth 1, 2, 3, …, 9, and 10 victory points (corresponding to the castle number).
- Whoever sends more soldiers to a given castle conquers that castle and wins its victory points.
- If you each send the same number of troops, you split the points equally.

Remember there are 55 points at stake in each 'battle'. This means that the sum of (i,j) and (j,i) should be 55. Check that this holds for your finished matrix.

The matrix will contain the value `22.5` along the diagonal (because you are of course evenly matched with yourself as an oponent). 
___

The player with the submission in the original row 0 in the data (who is therefore also at row 0 in the score matrix) has the following scores: 

`[22.5, 27. , 17. , ..., 22. , 21. , 30. ]`

In [ ]:
results = np.zeros((1321, 1321))
points = np.array(range(1, 11))
for i in range(data.shape[0]):
    for j in range(data.shape[0]):
        # compute points of i against player j
        results[i][j] = sum(points[data[i] > data[j]]) + sum(points[data[i] == data[j]] / 2)

In [ ]:
for i in range(1321):
    for j in range(1321):
        if results[i][j] + results[j][i] != 55:
            print('non equal')
print('all good')

### 13.9. Calculate the number of wins, ties, and losses for each of the submissions
 
The result should be a 1321x3 matrix with the three columns `[wins ties losses]`.
___
Result for submission at row 0: `[754, 17, 550]`

In [ ]:
stats = []
for i, results in enumerate(results):
    w = t = l = 0
    for r in results:
        if r > 27.5:
            w+=1
        elif r < 27.5:
            l+=1
        else: 
            t+=1
    stats.append([w, t, l])

stats = np.array(stats)

In [ ]:
stats[0]

### 13.10. Plot the histogram based on the number of wins per submission
Hint: Check the lecture jupyter notebook

In [ ]:
fig, ax = plt.subplots()
ax.hist(stats[:, 0]);

### 13.11. Plot the histogram based on the number of losses per submission
Hint: Check the lecture jupyter notebook


In [ ]:
fig, ax = plt.subplots()
ax.hist(stats[:, -1]);

### 13.12. Check if the distributions in exercise 13.10 and 13.11 follow a normal distribution
Hint: Make QQ plots (check the lecture notebook)

In [ ]:
# use same code on other data

### 13.13. Find the top 5 submissions based on their number of wins
Hint: https://numpy.org/doc/stable/reference/generated/numpy.argsort.html

Find the indices of the 5 submissions with the highest number of wins.
___
Indices: `609, 805, 178, 1222, 470`

In [ ]:
indices = list(reversed(np.argsort(stats[:, 0])[-5:]))

for i, index in enumerate(indices):
    print(f'Submission #{indices[i]} won {stats[index][0]}: {data[index]}')